In [13]:
include("./src/structs.jl")
include("./src/scalar_operators.jl")
include("./src/broadcast_operators.jl")
include("./src/graph.jl")
include("./src/forward.jl")
include("./src/backward.jl")
include("./src/data_loader.jl")
include("./src/nn_model.jl")

swish (generic function with 1 method)

In [3]:
using JLD2
X_train = load("./data/imdb_dataset_prepared.jld2", "X_train")
y_train = load("./data/imdb_dataset_prepared.jld2", "y_train")
X_test = load("./data/imdb_dataset_prepared.jld2", "X_test")
y_test = load("./data/imdb_dataset_prepared.jld2", "y_test")
nothing

println("Data loaded successfully.")
println("X_train size: ", size(X_train))
println("y_train size: ", size(y_train))
println("X_test size: ", size(X_test))
println("y_test size: ", size(y_test))

Data loaded successfully.
X_train size: (17703, 8000)
y_train size: (1, 8000)
X_test size: (17703, 2000)
y_test size: (1, 2000)


In [5]:
dataset = DataLoader((X_train, y_train), batchsize=64, shuffle=true)

DataLoader{Tuple{LinearAlgebra.Adjoint{Float32, Matrix{Float32}}, BitMatrix}, Random.TaskLocalRNG}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[1 1 … 0 0]), 8000, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  7991, 7992, 7993, 7994, 7995, 7996, 7997, 7998, 7999, 8000], 64, 125, true, Random.TaskLocalRNG())

In [15]:
input_neurons = size(X_train, 1)

17703

In [17]:
model = Chain(
    Dense((input_neurons => 32), ReLU, name="ReLU_dense"),
    Dense((32 => 1), σ, name="σ_dense")
)

Chain{Tuple{Dense{typeof(ReLU)}, Dense{typeof(σ)}}}((Dense{typeof(ReLU)}(Variable(Float32[-0.012824612 0.016066525 … 0.011139176 0.013078813; 0.009789155 0.008216373 … -0.0060159015 -0.0026054694; … ; -0.0088105155 -0.00955435 … -0.0002748799 -0.00974984; 0.001834291 0.009042097 … -0.017506108 -0.0053298576], nothing, "ReLU_dense"), nothing, Main.ReLU), Dense{typeof(σ)}(Variable(Float32[0.2760543 -0.23962979 … 0.14839661 0.019258803], nothing, "σ_dense"), nothing, Main.σ)))

In [18]:
loss(m, x, y) = binary_cross_entropy(m(x), y)
accuracy(m, x, y) = mean((m(x) .> 0.5) .== (y .> 0.5))

accuracy (generic function with 1 method)

In [1]:
graph = loss(model, x, y)
sorted_graph = topological_sort(graph)

LoadError: UndefVarError: `loss` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [2]:
epochs = 5

5

In [3]:
function test(set)
    L = 0.0
    for j = set
      x.output .= inputs[j,:]
      y.output .= targets[j,:]
      L += forward!(graph)
    end
    return L / length(set) 
end

test (generic function with 1 method)

In [ ]:
for i=1:epochs
    train!(dataloader[1:64], model, graph, η)
    test(test_set)
end

In [ ]:
function train!(batch, model, graph, lr=0.01f0)
    j = first(batch)
    x.output .= inputs[j,:]
    y.output .= targets[j,:]
    forward!(graph)
    backward!(graph)

    grads = gradient(model)

    for j in Iterators.drop(batch, 1)
        x.output .= inputs[j,:]
        y.output .= targets[j,:]

        forward!(graph)
        backward!(graph)
        
        accumulate_gradients!(grads, gradient(model))
    end

    update_params!(model, lr; grads, batch_len=length(batch))
    forward!(graph)
    return nothing
end

In [ ]:
for epoch in 1:10
    avg_loss = train_epoch!(model, dataloader, cross_entropy_loss, opt, graph)
    println("Epoch $epoch: Loss = $avg_loss")
end